In [ ]:
import sys
sys.path.append('../src')

import ipywidgets as widgets
import pandas as pd
import qiskit
import math

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, BasicAer, Aer, transpile, assemble
from qiskit.tools.visualization import plot_histogram

from Crypto.Hash import Poly1305
from Crypto.Cipher import AES
from binascii import unhexlify

import numpy as np
from numpy.random import randint
import panel as pn

import qkd
pn.extension()
pn.extension('terminal')

import time

In [ ]:
Aer.backends()

In [ ]:
dashboard = None
mac = None

In [ ]:
temp_backend = Aer.get_backend("qasm_simulator")
temp_air_sim = temp_backend

In [ ]:
# Parameter selectors
n_bits = pn.widgets.IntInput(name="Number of Qubits", value=10, step=1, start=10, end=1000000)
displacement = pn.widgets.FloatSlider(name="Distance (km)", value=0, step=1, start=0, end=20)
#% to db: db = 10 log(1+%)
#db to %:%=(10^db/10)*100
print(1-1/pow(10,(displacement.value*0.4+2)/10))
print(1-1/pow(10,(displacement.start*0.4+2)/10))
print(1-1/pow(10,(displacement.end*0.4+2)/10))
decoherence = pn.widgets.FloatSlider(name="Decoherence rate", value=1-1/pow(10,(displacement.value*0.4+2)/10), step=0.01, start=1-1/pow(10,(displacement.start*0.4+2)/10), end=1-1/pow(10,(displacement.end*0.4+2)/10))
timesl = pn.widgets.FloatSlider(name="Time to establish first key (km)", value=488-(7.19*displacement.value)+(1.5*pow(displacement.value,2)), step=5, start=488-(7.19*displacement.start)+(1.5*pow(displacement.start,2)), end=488-(7.19*displacement.end)+(1.5*pow(displacement.end,2)))
receiver = pn.widgets.Select(name="Target", options=['Bob', 'Test'])
## Greesan --> more inputs Code for  qprot, eavesdropping, and solutions
qprotgroup = pn.widgets.RadioButtonGroup(
    name='cb_qprot', options=['BB84','Ekert'])
eavebox = pn.widgets.Checkbox(name='Eavesdropping')
eaveslide = pn.widgets.FloatSlider(name="Leftover Eavesdropping rate", value=0, step=0.01, start=0, end=1)
#eaveslide = pn.widgets.FloatSlider(name="Leftover Eavesdropping rate", value=0, step=0.01, start=0, end=1-decoherence.value)
#eaveslide.link(decoherence,'value')
isolgroup = pn.widgets.CheckButtonGroup(
    name='cb_i_soln', options=['SDN', 'Poly1305-AES'])
isol2 = pn.widgets.CheckButtonGroup(
    name='cb_i_soln', options=['Cascade'])
updatebutton = pn.widgets.Button(name="Update Inputs",button_type='primary')
dispreval = displacement.value
decopreval = decoherence.value
eaveprebool = eavebox.value
eavepreval = eaveslide.value
timepreval = timesl.value

In [ ]:
keys = []
bases = []
encoded_output = []

bob_results = []
# Generate key
def generate_key(event=None):
    """
    Function for generating a random key and bases of length n.
    
    :param n: length of bitstring to generate
    """
    n = n_bits.value
    bit_key = randint(2, size=n)
    bit_basis = randint(2, size=n)
    
    keys.append(bit_key)
    bases.append(bit_basis)
    
    if select_auth.value == 'Poly1305':
        secret = b'Thirtytwo very very secret bytes'
        
        global mac
        mac = Poly1305.new(key=secret, cipher=AES)
        mac.update(b'Hello')
        print("Nonce: ", mac.nonce.hex())
        print("MAC:   ", mac.hexdigest())

        time.sleep(0.25)
        terminal.write("Nonce: " + str(mac.nonce.hex()) + "\n")
        time.sleep(0.25)
        terminal.write("MAC:   " + str(mac.hexdigest()) + "\n")
        time.sleep(0.25)
    
    
    
    terminal.write("\nGenerating " + str(n) + "-qubit key and basis ")
    for i in range(3):
        time.sleep(0.25)
        terminal.write(".")
    time.sleep(0.25)
    terminal.write(" Finished")
    time.sleep(0.4)
    terminal.write("\nAlice's Key: " + str(bit_key) + "\nAlice's Encoding Basis: " + str(bit_basis))
    
generate_button = pn.widgets.Button(name="Generate key")
generate_button.on_click(generate_key)

qubit_select = pn.widgets.Select(name="Qubit", options=[])

measure_button = pn.widgets.Button(name="Measure")
### Write code for key with eavesdropper with measure
# Outputs
terminal = pn.widgets.Terminal("Welcome to the QKD Simulator.\n\n", height=600, sizing_mode='stretch_width', options={"cursorBlink": True})
clear_terminal = pn.widgets.Button(name="Clear terminal")

send_button = pn.widgets.Button(name="Send")

measure_terminal = pn.widgets.Terminal(height=120, sizing_mode='stretch_width', options={"cursorBlink": True})

def send(event=None):
    #msg_receiver = receiver.value
    
    terminal.write("\nEncoding ")
    encode()
    for i in range(3):
        time.sleep(0.25)
        terminal.write(".")
    time.sleep(0.25)
    terminal.write(" Sent")
    time.sleep(0.4)
    
def encode(event=None):
    """
    Function for encoding a message.
    
    :param bit_key: Randomly generated bitstring key
    :param bit_bases: Bases for each bit in bit_key
    """
    bit_key = keys[-1]
    bit_bases = bases[-1]
    output = []
    
    # length of bit_key and bit_bases should be the same
    assert len(bit_key) == len(bit_bases), "Key and bases sequence should be equivalent."
    
    for i in range(len(bit_key)):
        qc = QuantumCircuit(1, 1)
        
        # Encode qubit in Z-basis (horizontal-vertical)
        if bit_bases[i] == 0:
            if bit_key[i] == 0:
                pass
            else:
                qc.x(0)
        
        # Encode qubit in X-basis (diagonal)
        else:
            if bit_key[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
                
        qc.barrier()
        output.append(qc)
        
    encoded_output.append(output)
    qubit_select.options += ["Qubit " + str(i) for i in range(len(output))]

send_button.on_click(send)
    
def delete_text(event=None):
    terminal.clear()

clear_terminal.on_click(delete_text)

In [ ]:
def measure_qubits(event=None):
    measure_terminal.clear()
    measure_terminal.write("\nMeasuring ")
    for i in range(3):
        time.sleep(0.25)
        measure_terminal.write(".")
    time.sleep(0.25)
    measure_terminal.write(" Finished")
    time.sleep(0.4)
    
    noise = decoherence.value
    measure_terminal.write(noise)
    msg = encoded_output[-1]
    n = n_bits.value
    bases = randint(2, size=n)
    
    if noise > 0:
        
        #temp_backend = Aer.get_backend("aer_simulator")
        temp_results = []
        
        noise_idx = []
        for i_n in range(int(len(msg)*noise)):
            noise_idx.append(random.randint(0, len(msg)-1))
            
        for n_idx in noise_idx:
            # Z-basis
            if bases[n_idx] == 0: 
                msg[n_idx].measure(0, 0)

             # X-basis
            if bases[n_idx] == 1:
                msg[n_idx].h(0)
                msg[n_idx].measure(0, 0)
                
            #temp_aer_sim = Aer.get_backend("aer_simulator")
            temp_qobj = assemble(msg[n_idx], shots=1, memory=True)
            temp_sim_results = temp_aer_sim.run(temp_qobj).result()
            temp_measured_bit = int(temp_sim_results.get_memory()[0])
            temp_results.append(temp_measured_bit)
            
    secret = b'Thirtytwo very very secret bytes'
    mac = Poly1305.new(key=secret, cipher=AES)
    mac.update(b'Hello')
    print("Nonce: ", mac.nonce.hex())
    print("MAC:   ", mac.hexdigest())
    
    passed = False
    msg_ = b"I am Alice."
    
    nonce_hex = mac.nonce.hex()
    mac_tag_hex = mac.hexdigest()

    secret = b'Thirtytwo very very secret bytes'
    nonce = unhexlify(nonce_hex)
    mac = Poly1305.new(key=secret, nonce=nonce, cipher=AES, data=msg_)
    try:
        mac.hexverify(mac_tag_hex)
        print("\nThe message '%s' is authentic" % msg_)
        passed = True
    except ValueError:
        print("\nThe message or the key is wrong")
        passed = False
         
    backend = Aer.get_backend("aer_simulator")
    results = []
    
    for i in range(len(msg)):
        # Z-basis
        if bases[i] == 0: 
            msg[i].measure(0, 0)
            
         # X-basis
        if bases[i] == 1:
            msg[i].h(0)
            msg[i].measure(0, 0)
            
        aer_sim = Aer.get_backend("aer_simulator")
        qobj = assemble(msg[i], shots=1, memory=True)
        sim_results = aer_sim.run(qobj).result()
        measured_bit = int(sim_results.get_memory()[0])
        results.append(measured_bit)
        
    bob_results.append(results)
    
    
    if passed == True:
        alice_msg = str("\nThe message is authentic.")
        measure_terminal.write(alice_msg + "\n")
        measure_terminal.write("\nGenerated Key: "+ str(results))
    
    else:
        measure_terminal.write("\nThe message or the key is wrong. Terminating ")
        for i in range(3):
            time.sleep(0.25)
            measure_terminal.write(".")
        
        measure_terminal.write("\nSession ended.")
        


measure_button.on_click(measure_qubits)

select_auth = pn.widgets.Select(name="Authentication Protocol", options=["Poly1305", "AES", "ZKP"])

pre_shared = pn.widgets.TextInput(name="Pre-Shared Key", value='Thirty two very very secret bytes')

auth_input = pn.widgets.TextInput(name="Authentication Tag", value='Thirty two very very secret bytes')

#updash()

measurement = pn.Row(pn.WidgetBox(auth_input, measure_button, height=120), measure_terminal)

In [ ]:
def generate_key2(n):
    """
    Function for generating a random key and bases of length n.
    
    :param n: length of bitstring to generate
    """
    bit_key = randint(2, size=n)
    bit_basis = randint(2, size=n)
    
    return bit_key, bit_basis

# Code for measuring
def encode2(bit_key, bit_bases):
    """
    Function for encoding a message.
    
    :param bit_key: Randomly generated bitstring key
    :param bit_bases: Bases for each bit in bit_key
    """
    output = []
    
    # length of bit_key and bit_bases should be the same
    assert len(bit_key) == len(bit_bases), "Key and bases sequence should be equivalent."
    
    for i in range(len(bit_key)):
        qc = QuantumCircuit(1, 1)
        
        # Encode qubit in Z-basis (horizontal-vertical)
        if bit_bases[i] == 0:
            if bit_key[i] == 0:
                pass
            else:
                qc.x(0)
        
        # Encode qubit in X-basis (diagonal)
        else:
            if bit_key[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
                
        qc.barrier()
        output.append(qc)
        
    return output

In [ ]:
def encodeBits(abits,abasis):
    s = "["
    qc = QuantumCircuit(len(abits))
    for i in range(len(abits)):
        if abits[i] == 0:
            if abasis[i] == 0:
                s+='0'
            else:
                qc.x(i)
                s+='1'
        else:
            qc.x(i)
            if abasis[i] == 0:
                s+='+'
            else:
                qc.h(i)
                s+='-'
        if i < len(abits)-1:
            s+= " "
    qc.barrier()
    return (qc,s+"]")

def decomeasure(perc,qc,basis):
    meascnt = 0
    measgate = QuantumCircuit(1, name='MEASURE')
    measgate.h(0)
    for i in range(len(basis)):
        print("meas",i)
        if randint(0,100) < perc*100:
            print(i,"is measured")
            qc.append(measgate,[i])
            meascnt+=1
    qc.barrier()
    return (qc,meascnt)

def eavesdrop(perc,qc,basis):
    eavecnt = 0
    eavesucc = 0
    eavegate = QuantumCircuit(1, name='EAVESDROP')
    for i in range(len(basis)):
        if randint(0,100) < perc*100:
            eavecnt+=1
            qc.append(eavegate,[i])
            r = randint(0,1)
            if basis[i] == r:
                eavesucc+=1
            if r == 1:
                qc.h(i)
    qc.barrier()
    return (qc,eavecnt,eavesucc)

def bobmeasure(qc,bobbasis):
    for i in range(len(bobbasis)):
        if bobbasis[i] == 1:
            qc.h(i)
    return qc

In [ ]:
def genrand(n):
    bits = randint(2, size=n)
    return bits

In [ ]:
bb84flag = False
Ekertflag = False
def on_BB84(eaves,eaveprebool,eavesperc,eavepreval,decoperc,decopreval,event=None):
    global bb84flag
    global Ekertflag
    if not bb84flag or eavesperc!=eavepreval or decoperc!=decopreval:
        terminal.clear()
        terminal.write("BB84\n")
        (alicebits,alicebasis) = generate_key2(n_bits.value)
        terminal.write("Alice Bits:               " + str(alicebits) + "\n")
        terminal.write("Alice Encoding Basis:     " + str(alicebasis) + "\n")
        (qc,s) = encodeBits(alicebits,alicebasis)
        terminal.write("Alice Bits After Encoding:" + s + "\n")
        if decoperc>0:
            qc,decoherecnt = decomeasure(decoperc,qc,alicebasis)
        terminal.write(str(decoherecnt) + " bits lost due to noise")
        if eaves and eavesperc>0:
            qc,eavecnt,eavesucc = eavesdrop(eavesperc,qc,alicebasis)
            
        bobbasis = genrand(n_bits.value)
        terminal.write("\nBob Decoding Basis:       " + str(bobbasis) + "\n")
        bobmeasure(qc,bobbasis)
        terminal.write(qc.draw(output="text"))
        terminal.write("\n")
    bb84flag = True
    Ekertflag = False
    #inputs: coherence
    #inputs: #bits
    #outputs:

In [ ]:
def on_Ekert(event=None):
    global bb84flag
    global Ekertflag
    if not Ekertflag:
        terminal.clear()
        terminal.write("Ekert91\n")
        print("")
    bb84flag = False
    Ekertflag = True

In [ ]:
def on_eavesdrop(event=None):
    a = True
    #ouputs ->reveal eavesdropping ratio --> terminal displays eavesdropping content

In [ ]:
def on_SDN(event=None):
    a = True

In [ ]:
def on_Poly1305AES(event=None):
    a = True

In [ ]:
def on_Cascade(event=None):
    a = True

In [ ]:
def updash(event=None):
    global dashboard
    dashboard = pn.Row(pn.WidgetBox(pre_shared, n_bits, decoherence, displacement, timesl, qprotgroup, eavebox, eaveslide, isolgroup, isol2, updatebutton, generate_button, clear_terminal, send_button, height=600),
    terminal)
updash()

In [ ]:
def update(event=None):
    global isolgroup
    global eavebox
    global eaveslide
    global timesl
    global displacement
    global decoherence
    global decopreval
    global dispreval
    global timepreval
    global eaveprebool
    global eavepreval
    #print(eavebox)
    #print(eavebox.value)
    
    if eavebox.value:
        #isolgroup = pn.widgets.CheckButtonGroup(name='cb_i_soln', options=['SDN', 'Poly1305-AES', 'Cascade'])
        eaveslide.disabled = False
        if eaveslide.value > 0:
            isolgroup.disabled = False
        else:
            isolgroup.disabled = True
        if eaveslide.value>0 or decoherence.value>0:
            isol2.disabled = False
        else:
            isol2.disabled = True
        #eaveslide.end = 1-decoherence.value
        eavenoise = eaveslide.value + decoherence.value
        on_eavesdrop()
    else:
        #isolgroup = pn.widgets.CheckButtonGroup(name='cb_i_soln', options=['SDN', 'Poly1305-AES', 'Cascade'],disabled=True)
        eaveslide.disabled = True
        isolgroup.disabled = True
        if decoherence.value > 0:
            isol2.disabled = False
        else:
            isol2.disabled = True
        eavenoise = decoherence.value
    slideflag = True
    #coherence = displacement.value*0.4+2
    #time = 488-(7.19*displacement.value)+(1.5*displacement.value*displacement.value)
    #coh=1-1/pow(10,(displacement.value*0.4+2)/10)
    #disp = 
    
    if slideflag and decopreval != decoherence.value:
        displacement.value = (-2.5)*(10*math.log10(1-decoherence.value)+2)
        timesl.value = 488-(7.19*displacement.value)+(1.5*pow(displacement.value,2))
        print(displacement.value,timesl.value)
        slideflag = False
    if slideflag and dispreval != displacement.value:
        decoherence.value = 1-1/pow(10,(displacement.value*0.4+2)/10)
        timesl.value = 488-(7.19*displacement.value)+(1.5*pow(displacement.value,2))
        print(decoherence.value,timesl.value)
        slideflag = False
    if slideflag and timepreval != timesl.value:
        displacement.value = math.sqrt((timesl.value-479)*2/3)+2.4
        decoherence.value = 1-1/pow(10,(displacement.value*0.4+2)/10)
        print(displacement.value,decoherence.value)
        slideflag = False
    
    dispreval = displacement.value
    timepreval = timesl.value
    if qprotgroup.value == "Ekert":
        on_Ekert()
    else:
        on_BB84(eavebox.value,eaveprebool,eaveslide.value,eavepreval,decoherence.value,decopreval)
        decopreval = decoherence.value
        eaveprebool = eavebox.value
        eavepreval = eaveslide.value
    #print(isolgroup.value)
    if isolgroup.value and len(isolgroup.value)>0:
        if "SDN" in isolgroup.value:
            print("SDN Activated")
            on_SDN()
        if "Poly1305-AES" in isolgroup.value:
            print("AES Activated")
            on_Poly1305AES()
    #print(isol2.value)
    if isol2.value and len(isol2.value)>0 and isol2.value[0]:
        print("Cascade Activated")
        on_Cascade()
    #updash()
updatebutton.on_click(update)
        

In [ ]:
#isolgroup = pn.widgets.CheckButtonGroup(
#    name='cb_i_soln', options=['SDN', 'Poly1305-AES', 'Cascade'],disabled=True)
#dashboard = updash()
#dashboard

In [ ]:
#isolgroup
#eavebox
#qprotgroup

In [ ]:
update()
dashboard

In [ ]:
update()

In [ ]:
measurement

In [ ]:
# pn.Row(pn.WidgetBox(qubit_select, height=75), qubit_select, height=100)

In [ ]:
# static_text = pn.widgets.StaticText(value=qubit_select.options[0].draw())
# static_text

In [ ]:
# qubit_select.options[0].draw()

In [ ]:
if mac:
    mac.nonce.hex()

In [ ]:
mac

In [ ]:
# network table

network = pd.DataFrame(columns=["Alice", "Bob", "Public"])
network

In [ ]:
n = 100

alice_key, alice_basis = generate_key2(n)

In [ ]:
alice_key

In [ ]:
import time
import datetime

int(time.time()) // 1000

In [ ]:
# Allowed delay
delay_threshold = 10 * 60 # 10 minutes

# Alice time
t_alice = time.time()

# Alice seed
alice_init_seed = int(np.ceil(t_alice / delay_threshold))
alice_init_seed

In [ ]:
# Bob time
t_bob = time.time()

# Bob seed
bob_init_seed = int((t_bob+delay_threshold) / delay_threshold)
bob_init_seed

In [ ]:
def generate_seed(init_seed):
    return np.abs(hash(str(init_seed)) // (2**32 - 1))

In [ ]:
np.random.seed(generate_seed(alice_init_seed))

def generate_pad(n):
    return randint(2, size=n)

alice_otp = generate_pad(100)
alice_otp

In [ ]:
np.random.seed(generate_seed(bob_init_seed))

bob_otp = generate_pad(100)
bob_otp

In [ ]:
alice_otp == bob_otp